In [1]:
!pip install google-play-scraper
!pip install sastrawi
!pip install emoji

In [2]:
!pip install numpy==1.24.3 gensim==4.3.2
!pip install --upgrade scikit-learn

In [3]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from google_play_scraper import app, reviews, Sort, reviews_all
import emoji
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
import requests
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
scrapreview = reviews_all(
    'com.gojek.gopay',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
)

scrapreview = scrapreview[:5000]

In [5]:
with open('gopay_reviews.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrapreview:
        writer.writerow([review['content']])

In [6]:
# Misalkan scrapreview sudah diubah ke DataFrame
app_reviews_df = pd.DataFrame(scrapreview)
# Fungsi untuk memberi label berdasarkan skor
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

# Tambahkan kolom 'sentiment' ke DataFrame
app_reviews_df['sentiment'] = app_reviews_df['score'].apply(label_sentiment)

# Simpan ke CSV untuk memastikan hasilnya
app_reviews_df.to_csv('ulasan_dengan_label.csv', index=False)

# Cek jumlah data dan distribusi sentimen
print(f"Jumlah ulasan: {len(app_reviews_df)}")
print(app_reviews_df['sentiment'].value_counts())

Jumlah ulasan: 5000
sentiment
negatif    3168
positif    1459
netral      373
Name: count, dtype: int64


In [7]:
import pandas as pd
import re

# Misalkan data sudah di-load dari CSV hasil langkah sebelumnya
app_reviews_df = pd.read_csv('ulasan_dengan_label.csv')

def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = remove_emoji(text)  # Hapus emoji


    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def remove_stopwords(word_list):
    stopwords_indonesia = stopwords.words('indonesian')
    no_words = [w for w in word_list if w.lower() not in stopwords_indonesia]
    return no_words

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

def load_slangwords():
    slangwords = {}
    url = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Memastikan permintaan berhasil
        # File berformat seperti dictionary Python, jadi kita evaluasi isinya
        slangwords = eval(response.text)  # Konversi string ke dictionary
        return slangwords
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil slangword: {e}")
        return {}

slangwords = load_slangwords()

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

def preprocess(text):
    text = cleaningText(text)
    text = casefoldingText(text)
    tokens = tokenizingText(text)
    tokens = remove_stopwords(tokens)
    fixed_text = fix_slangwords(' '.join(tokens))
    return fixed_text

# Terapkan preprocessing
app_reviews_df['clean_text'] = app_reviews_df['content'].apply(preprocess)

# Hapus baris dengan teks kosong
app_reviews_df = app_reviews_df[app_reviews_df['clean_text'] != '']

app_reviews_df = app_reviews_df.dropna()

# Hapus duplikat
app_reviews_df = app_reviews_df.drop_duplicates(subset=['clean_text'])

# Simpan hasil
app_reviews_df.to_csv('ulasan_clean.csv', index=False)

# Cek beberapa baris
print(app_reviews_df[['clean_text', 'sentiment']].head())
print(f"Jumlah data setelah preprocessing: {len(app_reviews_df)}")

                                          clean_text sentiment
0  saldo kepotong top up game oke gapapa hitam be...   negatif
2  kenapa gak masuk pin digit beda hp hp yang ter...   negatif
3  kecewa sudah top up gak masuk masuk nunggu har...   negatif
4  gopay mengecewakansaya driver gojek pagi siang...   negatif
5  suka gopay murah saldo aman bikin kesel proses...   negatif
Jumlah data setelah preprocessing: 3472


In [8]:
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3472 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              3472 non-null   object
 1   userName              3472 non-null   object
 2   userImage             3472 non-null   object
 3   content               3472 non-null   object
 4   score                 3472 non-null   int64 
 5   thumbsUpCount         3472 non-null   int64 
 6   reviewCreatedVersion  3472 non-null   object
 7   at                    3472 non-null   object
 8   replyContent          3472 non-null   object
 9   repliedAt             3472 non-null   object
 10  appVersion            3472 non-null   object
 11  sentiment             3472 non-null   object
 12  clean_text            3472 non-null   object
dtypes: int64(2), object(11)
memory usage: 379.8+ KB


In [9]:
clean_df = pd.read_csv('ulasan_clean.csv')

# svm tf idf

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Load data yang sudah dibersihkan
clean_df = pd.read_csv('ulasan_clean.csv')

# Mapping label ke angka
label_map = {'negatif': 0, 'netral': 1, 'positif': 2}
clean_df['label'] = clean_df['sentiment'].map(label_map)

# Ekstraksi fitur TF-IDF
tfidf = TfidfVectorizer(max_features=3000)  # ambil fitur top 5000
X = tfidf.fit_transform(clean_df['clean_text']).toarray()
y = clean_df['label']

# Split data 80% train / 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Training SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Prediksi
y_pred_train_svm = svm_model.predict(X_train)
y_pred_test_svm = svm_model.predict(X_test)

# Evaluasi akurasi
accuracy_train_svm = accuracy_score(y_train, y_pred_train_svm)
accuracy_test_svm = accuracy_score(y_test, y_pred_test_svm)

# Tampilkan hasil
print("SVM - Accuracy Train:", accuracy_train_svm)
print("SVM - Accuracy Test:", accuracy_test_svm)

SVM - Accuracy Train: 0.8800864241987757
SVM - Accuracy Test: 0.8762589928057554


In [51]:
import random

# Menggunakan model untuk prediksi
y_pred_test_svm = svm_model.predict(X_test)  # Prediksi untuk data uji

# Menampilkan hasil prediksi secara acak
random_indices = random.sample(range(len(X_test)), 10)  # Pilih 10 indeks acak

print("Prediksi Sentimen untuk Data Uji (Acak):")
for i in random_indices:
    print(f"Text: {clean_df['clean_text'].iloc[i]}, Prediksi: {y_pred_test_svm[i]}")

Prediksi Sentimen untuk Data Uji (Acak):
Text: gopay ga nyediain namanya gopaypargopay pelajar ga ktp saja gunain namanya transfer butuh transfer semacamnya gopay gopay par kalau batasan lulus sekolahsudah memiliki ktp, Prediksi: 0
Text: ini iya saya top up itemku sudah benar tuh sudah terbaru aplikasinya saya bayar web habis web gak gopay nya saya manual hasilnya tidak tuh tombol transaksi ya sudah saya coba tetep gak ya sudah kayaknya gabisa deh emang bapuk ini aplikasi, Prediksi: 0
Text: murah harga pulsapaket promo nya murah rp selebihnya saja murah iya murah harga aslinyapromo murahnya sengaja download buktikan dimana letak murahnya gak marketing comedy, Prediksi: 0
Text: bug voucher gopay games aktif cashback cashbacknya masuk memenuhi sk yang berlaku voucher hilang voucher diluar gopay games, Prediksi: 0
Text: top up flass bca saldo ga masuk saldo sdh dipotong update saldo ga lapor bank bank disuruh lapor gopay nyaandingnya uninstall aplikasi sajaanggap saja duit ilangsangat rec

# rf word2vec

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import Word2Vec

# Load data bersih
data = pd.read_csv('ulasan_clean.csv')
label_map = {'negatif': 0, 'netral': 1, 'positif': 2}
data['label'] = data['sentiment'].map(label_map)

# Tokenizing untuk Word2Vec
data['tokens'] = data['clean_text'].apply(nltk.word_tokenize)

# Train Word2Vec model
w2v_model = Word2Vec(sentences=data['tokens'], vector_size=100, window=5, min_count=1, sg=1)

# Fungsi untuk merata-rata embedding setiap kalimat
def vector_average(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

# Konversi semua text jadi vektor
X_vectors = np.array([vector_average(tokens) for tokens in data['tokens']])
y = data['label']

# Split data
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42, stratify=y
)

# Training Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_w2v, y_train_w2v)

y_pred_train_rf = rf_model.predict(X_train_w2v)
y_pred_test_rf = rf_model.predict(X_test_w2v)

print("Random Forest - accuracy_train:", accuracy_score(y_pred_train_rf, y_train_w2v))
print("Random Forest - accuracy_test :", accuracy_score(y_pred_test_rf, y_test_w2v))

Random Forest - accuracy_train: 1.0
Random Forest - accuracy_test : 0.8705035971223022


In [42]:
print(clean_df['sentiment'].value_counts())
print(f"Training data: {len(X_train_w2v)}")
print(f"Testing data: {len(X_test_w2v)}")

sentiment
negatif    3043
netral      357
positif      72
Name: count, dtype: int64
Training data: 2777
Testing data: 695


In [43]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_w2v, y_pred_train_rf)

print(f"Data training balanced: {Counter(y_train_bal)}")

Data training balanced: Counter({0: 2434, 1: 2434, 2: 2434})


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Melatih model Random Forest dengan data yang seimbang
rf_model_bal = RandomForestClassifier(random_state=42)
rf_model_bal.fit(X_train_bal, y_train_bal)

# Prediksi pada data uji
y_pred_bal = rf_model_bal.predict(X_test_w2v)

# Evaluasi Model
print("Random Forest (Data Seimbang) - Classification Report:")
print(classification_report(y_pred_test_rf, y_pred_bal))
print("Akurasi (Test Data):", accuracy_score(y_pred_test_rf, y_pred_bal))


Random Forest (Data Seimbang) - Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       689
           1       0.08      0.60      0.14         5
           2       0.09      1.00      0.17         1

    accuracy                           0.93       695
   macro avg       0.39      0.85      0.43       695
weighted avg       0.99      0.93      0.96       695

Akurasi (Test Data): 0.9338129496402877


In [49]:
# Menggunakan model untuk prediksi
y_pred_bal = rf_model_bal.predict(X_test_w2v)  # Prediksi untuk data uji

# Menampilkan hasil prediksi secara acak
random_indices = random.sample(range(len(X_test_w2v)), 10)  # Pilih 10 indeks acak

print("Prediksi Sentimen untuk Data Uji (Acak):")
for i in random_indices:
    print(f"Text: {clean_df['clean_text'].iloc[i]}, Prediksi: {y_pred_bal[i]}")

Prediksi Sentimen untuk Data Uji (Acak):
Text: saldo ga diakses upgrade gopay plus cek gopay tabungan rupiah tulisannya saldo transfer gopay tabungan maksudnya maksa nabung situ diakses tunggu minggu diakses, Prediksi: 0
Text: gopay aneh war diskon sudah dapat giliran bayar kendala sudah coba tetep gabisa hadehh ngadain event yang benar sudah gitu cepat habis gilir dapat eh ga bayar aneh mendeteksi aktivitas gak wajar aneh gak wajar coba dapat diskon nya apk nya bilang aktivitas gak wajar kalau gak iklas memberikan mah gak ngadain event kocak dehh dasar, Prediksi: 0
Text: top up gopay nominal masuk² ajukan bantuan respon informasi kendala sistem sistem libur menu bantuan seharus nya beranda biar pengguna tahu gopay libur apalah informasi taruh menu bantuan tolong dibenahi sistem nya kayak gini tiwas top up gopay ajar, Prediksi: 0
Text: gopay transaksi limit an juta perbulan nya tolak mengaktifkan gopay pinjam alasan nya karena foto ktp huruf hilang bukti transaksi limit segitu gopay ad

# LR TFIDF

In [37]:
# Import yang diperlukan
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Ekstraksi fitur menggunakan TF-IDF
tfidf = TfidfVectorizer(max_features=3000)  # Ambil 3000 fitur teratas
X = tfidf.fit_transform(clean_df['clean_text']).toarray()
y = clean_df['label']

# Pembagian data 80/20 (80% untuk latih, 20% untuk uji)
X_train_8020, X_test_8020, y_train_8020, y_test_8020 = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Logistic Regression
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_8020, y_train_8020)

# Prediksi dengan Logistic Regression
y_pred_lr = lr_model.predict(X_test_8020)

# Evaluasi Logistic Regression
print("Logistic Regression - Classification Report:")
print(classification_report(y_test_8020, y_pred_lr))
print("Logistic Regression - accuracy_train:", accuracy_score(y_train_8020, lr_model.predict(X_train_8020)))
print("Logistic Regression - accuracy_test:", accuracy_score(y_test_8020, y_pred_lr))


Logistic Regression - Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       612
           1       0.00      0.00      0.00        67
           2       0.00      0.00      0.00        16

    accuracy                           0.88       695
   macro avg       0.29      0.33      0.31       695
weighted avg       0.78      0.88      0.82       695

Logistic Regression - accuracy_train: 0.8797263233705438
Logistic Regression - accuracy_test: 0.879136690647482


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [46]:
import random

# Menggunakan model untuk prediksi
y_pred_lr = lr_model.predict(X_test_8020)  # Prediksi untuk data uji

# Menampilkan hasil prediksi secara acak
random_indices = random.sample(range(len(X_test_8020)), 10)  # Pilih 10 indeks acak

print("Prediksi Sentimen untuk Data Uji (Acak):")
for i in random_indices:
    print(f"Text: {clean_df['clean_text'].iloc[i]}, Prediksi: {y_pred_lr[i]}")

Prediksi Sentimen untuk Data Uji (Acak):
Text: aplikasi gak jelasapllikasi masalahdikit mendeteksi yang gakwajargakbisa melanjutkan pembayaranaplikasi error, Prediksi: 0
Text: curang beli pulsa diskon rp kasih pulsa saya coba rp eh sengaja ambil kesalahan mohon perbaiki, Prediksi: 0
Text: gopaylater ku diblokir peninjauan kunjung pulih sistem login aman setiaap login verifikasi muka keaaman khawatir dompet dicuri, Prediksi: 0
Text: top up emoney masuk saldo gopay terpotong rp ke saldo emoney masuk top up pulsa tanggal masuk jam masuk statusnya menunggu, Prediksi: 0
Text: aplikasi update busuk ganti devise login akun gopay disuruh verifikasi wajah gopay pakai ktp teman jakarta login akunnya tolong dihilangin fitur verifikasi muka nya ditambahi opsi off, Prediksi: 0
Text: mengalami isi saldo gopay ngak masuk gopay bri laporan terkirim saldo kgk masuk hati penipuan, Prediksi: 0
Text: paylater pakai kredivo shopee gopaylater terlambat sehari saja sudah langsung gak teman teman yang pakai p